In [4]:
import pandas as pd
import numpy as np
import mygene
import joblib

mg = mygene.MyGeneInfo()

In [5]:
import graph_tool as gt


def create_ppi_graph(edges_df: pd.DataFrame):
    edges_df = edges_df.loc[edges_df.iloc[:, 0] != edges_df.iloc[:, 1], :]

    nodes = list(set(edges_df.iloc[:, 0].unique().tolist() + edges_df.iloc[:, 1].unique().tolist()))

    edges_df.loc[:, 'source'] = edges_df.iloc[:, 0].apply(lambda x: nodes.index(x) if x in nodes else np.nan)
    edges_df.loc[:, 'target'] = edges_df.iloc[:, 1].apply(lambda x: nodes.index(x) if x in nodes else np.nan)

    edges_df = edges_df.dropna()

    edges_df.loc[:, 'union'] = edges_df.apply(lambda x: ", ".join(list(set([str(x['source'])]).union(set([str(x['target'])])))), axis=1)
    edges_df.loc[:, 'union'] = edges_df.apply(lambda x: ", ".join(list(set([str(x['source'])]).union(set([str(x['target'])])))), axis=1)
    edges_df = edges_df.drop_duplicates(subset=['union'])

    ppi_G = gt.Graph()

    for node in nodes:
        v = ppi_G.add_vertex()

    for index, row in edges_df.iterrows():
        ppi_G.add_edge(row['source'], row['target'], add_missing=False)

    print('Number of nodes:', ppi_G.num_vertices(), 'Number of edges:', ppi_G.num_edges())
    return ppi_G, nodes

In [7]:
large = pd.read_csv('../0_data/ppi_networks/large.txt', sep="\t", header=None).drop([0, 2], axis=1)
large.to_csv('large_network.csv')
large_ppi, large_nodes = create_ppi_graph(large)
large_ppi = {'network': large_ppi, 'nodes': large_nodes}
joblib.dump(large_ppi, 'large_ppi.sav')

Number of nodes: 16470 Number of edges: 233957


['large_ppi.sav']